In [1]:
#import useful libraries for analysis and modeling
import pandas as pd
import numpy as np
from sklearn import tree
from scipy import stats
import os
import pickle
import sys
import tempfile
import math
# import matplotlib as mpl
# import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.tree import export_graphviz, DecisionTreeClassifier
pd.options.mode.chained_assignment = None
from IPython.display import display, HTML
import warnings
import json
from matplotlib import cm  # color map
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
import copy as cp
import shap
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, DistanceMetric
from sklearn.preprocessing import scale
from sklearn.datasets import load_iris, load_wine
from matplotlib import pyplot as plt
import colorcet as cc
from sklearn.covariance import EllipticEnvelope 
#
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
from sklearn.cluster import HDBSCAN
from sklearn.cluster import AgglomerativeClustering
import gower
from scipy.spatial.distance import hamming

# Filter all warnings
warnings.filterwarnings("ignore")


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [17]:
# # list of 8 applications in dataset
classes = ['Dropcam', 'HP Printer', 'Netatmo Welcome', 'Withings Smart Baby Monitor', 'Netatmo weather station',\
           'Smart Things', 'Amazon Echo', 'Samsung SmartCam','TP-Link Day Night Cloud camera', 'Triby Speaker',\
              'Belkin Wemo switch', 'TP-Link Smart plug', 'PIX-STAR Photo-frame','Belkin wemo motion sensor',\
                     'Samsung Galaxy Tab', 'NEST Protect smoke alarm', 'Withings Smart scale', 'IPhone',\
                            'MacBook', 'Withings Aura smart sleep sensor','Light Bulbs LiFX Smart Bulb',\
                            'Blipcare Blood Pressure meter','iHome', 'Insteon Camera', 'Android Phone', 'Laptop']
classes_df = pd.DataFrame(classes, columns=['class'])

# list of all extracted features
feats_all = ["ip.len","ip.ttl","tcp.flags.syn","tcp.flags.ack","tcp.flags.push","tcp.flags.fin","tcp.flags.rst",\
            "tcp.flags.ece","ip.proto","srcport","dstport","ip.hdr_len","tcp.window_size_value","tcp.hdr_len","udp.length",\
            "Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Total","UDP Len Min","UDP Len Max",\
                "Flow IAT Min","Flow IAT Max","Flow IAT Mean","Flow Duration",\
                    "SYN Flag Count","ACK Flag Count","PSH Flag Count","FIN Flag Count","RST Flag Count","ECE Flag Count"]

# list of easy to compute online features - without means
feats_easy = ["ip.len","ip.ttl","tcp.flags.syn","tcp.flags.ack","tcp.flags.push","tcp.flags.fin","tcp.flags.rst",\
            "tcp.flags.ece","ip.proto","srcport","dstport","ip.hdr_len","tcp.window_size_value","tcp.hdr_len","udp.length",\
            "Min Packet Length","Max Packet Length","Packet Length Total","UDP Len Min","UDP Len Max",\
                "Flow IAT Min","Flow IAT Max","Flow Duration","SYN Flag Count","ACK Flag Count",\
                    "PSH Flag Count","FIN Flag Count","RST Flag Count","ECE Flag Count"]

feats_no_time = ["ip.len","ip.ttl","tcp.flags.syn","tcp.flags.ack","tcp.flags.push","tcp.flags.fin","tcp.flags.rst",\
            "tcp.flags.ece","ip.proto","srcport","dstport","tcp.window_size_value","tcp.hdr_len","udp.length",\
            "Min Packet Length","Max Packet Length","Packet Length Total",\
                "SYN Flag Count","ACK Flag Count","PSH Flag Count","FIN Flag Count","RST Flag Count","ECE Flag Count"]

In [18]:
# Define a function to check the conditions and assign values
def assign_sample_nature(row):
    if (row["Min Packet Length"] == -1 and
        row["Max Packet Length"] == -1 and
        row["Flow IAT Min"] == -1 and
        row["Flow IAT Max"] == -1):
        return "pkt"
    else:
        return "flw"

In [19]:
def read_csv(csv_file_names):
    csv_files = []
    for f_name in csv_file_names:
        df = pd.read_csv(f_name, sep=';')
        csv_files.append(df)
        
    return pd.concat(csv_files)

In [20]:
def plot_line(x_axis_values, y_axis_values, labels, x_title, y_title, fig_title):
    fig = plt.figure(figsize = (15, 5))
    colors= {'#6E7E99', '#ffbb78','#8A9961','#1f77b4', '#E06B5A', '#A95C68'}
    # for x_axis_val, y_axis_val, label, color in zip(x_axis_values, y_axis_values, labels, colors):
    #     plt.plot(x_axis_val, y_axis_val, label=label, color=color, linewidth=1.5)
    
    for x_axis_val, y_axis_val, color in zip(x_axis_values, y_axis_values, colors):
        plt.plot(x_axis_val, y_axis_val, 'g*-', color=color, linewidth=1.5, markersize=4)
    plt.plot(x_axis_val, [0.7 for i in range(0, len(x_axis_val))], color='gray', linestyle='--', lw=1)
    plt.plot(x_axis_val, [0.5 for i in range(0, len(x_axis_val))], color='gray', linestyle='--', lw=1)
    plt.xlabel(x_title, fontsize=15, labelpad=13)
    plt.ylabel(y_title, fontsize=15, labelpad=14)
    plt.xticks(rotation=90)
    plt.title(fig_title)
    plt.legend()
    plt.show()

In [21]:
def calculate_score(support_total, mult_score_support, score):
    macro_score = np.mean(np.array(score))
    weighted_score = np.sum(mult_score_support)/support_total
    
    return macro_score, weighted_score

In [22]:
cl_results_2_models_together = pd.read_csv('/home/nds-admin/UNSW_PCAPS/hyb_code/with_FL_Metric/cluster_model_analysis_results/score_after_clustering_2_models_together.csv')
cl_results_2_models_together = cl_results_2_models_together.sort_values(by='1_model_f1', ascending=False)

In [23]:
### The results coming from the first big model
cl_results_1st_model = pd.read_csv('/home/nds-admin/UNSW_PCAPS/hyb_code/with_FL_Metric/cluster_model_analysis_results/Proportional_fairness/unsw_models_PF_W_Cluster_Results_Most_Successful.csv')
cl_results_2nd_model = pd.read_csv('/home/nds-admin/UNSW_PCAPS/hyb_code/with_FL_Metric/cluster_model_analysis_results/second_model_cl_result_with_OtherAsLeastSucc.csv')

In [59]:
#### Create the list for clustering results using 2 different clustering path
results_from_1st_model = cl_results_1st_model[cl_results_1st_model['f1_score']>70]['Cluster_F1_Score'].to_list()
print(results_from_1st_model)
##
least_succ_classes_df = cl_results_1st_model[cl_results_1st_model['f1_score']<70]
least_succ_class_scores = pd.merge(right=least_succ_classes_df, left=cl_results_2nd_model, on='class').sort_values(by='f1_score_y', ascending=False)['Cluster_F1_Score_x'].to_list()
print(least_succ_class_scores)
##=
all_results_from_two_clustering = np.concatenate([np.array(results_from_1st_model), np.array(least_succ_class_scores)])
##
cl_results_2_models_together['score_2_clustering'] = all_results_from_two_clustering

[99.7902754015122, 99.42737145166085, 99.66301600673977, 97.5489726995085, 94.90475088520076, 96.68894699935156, 92.27665036924851, 92.79354886852713, 91.61994011527484, 79.6545640263724, 88.03270852347583, 92.23212017200456, 89.0788698982421, 74.22657757979907, 71.2702940103382, 89.05109489051092, 79.37955639652654]
[89.3879696438101, 87.95374393978543, 65.35155255193122, 87.38421916232218, 79.8287311302028, 93.98447462820675, 75.86196098780017, -1.0, -1.0]


In [73]:
##### SCORE CALCULATION #####
### With Unsuccessful Classes
score_calc_df = cl_results_2_models_together[cl_results_2_models_together['score_2_clustering']>1]
score_calc_df['mult_2_clustering'] = score_calc_df['score_2_clustering']*score_calc_df['support']
score_calc_df['mult_1_clustering'] = score_calc_df['Cluster_F1_Score']*score_calc_df['support']
score_calc_df['mult_First_model'] = score_calc_df['1_model_f1']*score_calc_df['support']
score_calc_df['mult_First_Second_model'] = score_calc_df['2_models_f1']*score_calc_df['support']
##
print('After the clustering with ALL - 2 clustering processes: ', calculate_score(np.sum(np.array(score_calc_df['support'])), score_calc_df['mult_2_clustering'], score_calc_df['score_2_clustering']))
print('After 1 clustering process using 2 models: ' ,calculate_score(np.sum(np.array(score_calc_df['support'])), score_calc_df['mult_1_clustering'], score_calc_df['Cluster_F1_Score']))
print('The overall score of the first big model: ' ,calculate_score(np.sum(np.array(score_calc_df['support'])), score_calc_df['mult_First_model'], score_calc_df['1_model_f1']))
print('The overall score after using 2 models: ' ,calculate_score(np.sum(np.array(score_calc_df['support'])), score_calc_df['mult_First_Second_model'], score_calc_df['2_models_f1']))


After the clustering with ALL - 2 clustering processes:  (87.80799626409801, 90.90327685246103)
After 1 clustering process using 2 models:  (79.5401744727372, 87.50424908710418)
The overall score of the first big model:  (79.23829498103625, 90.81743142574274)
The overall score after using 2 models:  (87.24061745427234, 92.96216220127778)


In [65]:
cl_results_2_models_together

,Unnamed: 0,class,1_model_f1,2_models_f1,Cluster_F1_Score,Cluster,support,score_2_clustering
0,0,Light Bulbs LiFX Smart Bulb,99.836551,99.836551,99.798226,5,1509.0,99.790275
1,1,Netatmo weather station,99.271827,99.271827,99.792919,5,248.0,99.427371
2,2,Smart Things,99.239209,99.239209,99.578059,5,592.0,99.663016
3,3,Samsung SmartCam,97.744553,97.744553,95.805455,6,13334.0,97.548973
4,4,Amazon Echo,96.684257,96.684257,94.592619,4,4649.0,94.904751
5,5,Belkin wemo motion sensor,95.942556,95.942556,95.306220,2,3282.0,96.688947
7,7,Laptop,94.084491,94.084491,91.759824,2,11484.0,92.276650
8,8,Insteon Camera,93.822807,93.822807,92.143589,2,6888.0,92.793549
9,9,iHome,93.602732,93.602732,88.000792,2,160.0,91.619940
10,10,Withings Smart scale,91.891892,91.891892,90.697478,2,6.0,79.654564
